#### 1st let's import our Libraries

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [7]:
!pip install lat-lon-parser
from lat_lon_parser import parse

### now let's get the cities of interest!
- Hamburg
- Munich
- San Francisco
- La Paz- Bolivia

In [9]:
url = 'https://www.wikipedia.org/wiki/Hamburg'
response = requests.get(url)
hamburg_soup = BeautifulSoup(response.content, 'html.parser')

hamburg_country = hamburg_soup.find(class_="infobox-data").get_text()
hamburg_latitude = hamburg_soup.find(class_="latitude").get_text()
hamburg_longitude = hamburg_soup.find(class_="longitude").get_text()

hamburg_country, hamburg_latitude, hamburg_longitude

('Germany', '53°33′N', '10°00′E')

### Creating a loop to get the cities!

In [11]:
cities = [ "Hamburg", "Munich", "San Francisco", "La Paz"]

countries = []
latitudes = []
longitudes = []

for city in cities:
  # get the soup for the city
  url = f"https://www.wikipedia.org/wiki/{city}"
  response = requests.get(url)
  city_soup = BeautifulSoup(response.content, 'html.parser')

  # extract the data
  city_country = city_soup.find(class_="infobox-data").get_text()
  city_latitude = city_soup.find(class_="latitude").get_text()
  city_longitude = city_soup.find(class_="longitude").get_text()

  # append data to a list
  countries.append(city_country)
  latitudes.append(city_latitude)
  longitudes.append(city_longitude)

### checking what we got

In [13]:
print(f"The cities are in the following countries: {countries}")
print(f"The cities have the following latitudes: {latitudes}")
print(f"The cities have the following longitudes: {longitudes}")

The cities are in the following countries: ['Germany', 'Germany', '\xa0United States', 'Bolivia']
The cities have the following latitudes: ['53°33′N', '48°08′15″N', '37°46′39″N', '16°29′45″S']
The cities have the following longitudes: ['10°00′E', '11°34′30″E', '122°24′59″W', '68°08′00″W']


### we need a table for the cities


In [15]:
cities_df = pd.DataFrame({"City_name": cities,
                          "Country": countries,
                          "Latitude": latitudes,
                          "Longitude": longitudes})

cities_df

,City_name,Country,Latitude,Longitude
0,Hamburg,Germany,53°33′N,10°00′E
1,Munich,Germany,48°08′15″N,11°34′30″E
2,San Francisco,United States,37°46′39″N,122°24′59″W
3,La Paz,Bolivia,16°29′45″S,68°08′00″W


### in order to add more cities in the future we should make our loop into a function and add the list we have to one!

In [17]:
cities = ["Hamburg", "Munich", "San Francisco", "La Paz"]

# create one single list to keep track of all values (instead of having separate lists)
city_data = []

for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    country = city_soup.find(class_="infobox-data").get_text()
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()

    # for each city we append a dictionary of values to the list
    city_data.append({"City_name": city,
                     "Country": country,
                     "Latitude": city_latitude,
                     "Longitude": city_longitude
                    })

cities_df = pd.DataFrame(city_data)
cities_df


,City_name,Country,Latitude,Longitude
0,Hamburg,Germany,53°33′N,10°00′E
1,Munich,Germany,48°08′15″N,11°34′30″E
2,San Francisco,United States,37°46′39″N,122°24′59″W
3,La Paz,Bolivia,16°29′45″S,68°08′00″W


changind latitude data type for the future

In [19]:
parse(hamburg_latitude)

53.55

## ok now loop to function!

In [21]:

def cities_dataframe(cities):

  city_data = []

  for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()
    country = city_soup.find(class_="infobox-data").get_text()

    # keep track of data per city
    city_data.append({"City_name": city,
                    "Country": country,
                    "Latitude": parse(city_latitude), # latitude in decimal format
                    "Longitude": parse(city_longitude), # longitude in decimal format
                    })

  return pd.DataFrame(city_data)


### does it work?

# cities_df

In [24]:
list_of_cities = [ "Hamburg", "Munich", "San Francisco", "La Paz"]

cities_df = cities_dataframe(list_of_cities)
cities_df

,City_name,Country,Latitude,Longitude
0,Hamburg,Germany,53.550000,10.000000
1,Munich,Germany,48.137500,11.575000
2,San Francisco,United States,37.777500,-122.416389
3,La Paz,Bolivia,-16.495833,-68.133333


## Let's find the population now

In [26]:
hamburg_population = hamburg_soup.find(string="Population").find_next("td").get_text()
hamburg_population

'1,964,021'

In [27]:
from datetime import datetime # to get today's date
from datetime import datetime, timedelta
from pytz import timezone

In [28]:
date = datetime.now().date()
cities = ["Hamburg", "Munich", "San Francisco", "La Paz"]

# create one list for population
population_data = []

for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

     # extract the relevant information
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = city_population.replace(",", "")
    today = datetime.today().strftime("%d.%m.%Y")

    # for each city we append a dictionary of values to the list
    population_data.append({"City": city,
                     "Population": int(city_population_clean),
                     "Population_Timestamp": date
                    })

population_data = pd.DataFrame(population_data)
population_data


,City,Population,Population_Timestamp
0,Hamburg,1964021,2024-12-04
1,Munich,1510378,2024-12-04
2,San Francisco,873965,2024-12-04
3,La Paz,755732,2024-12-04


### turn the loop into a function

import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime # to get today's date

In [100]:

def populations_dataframe(cities):

    population_data = []

    for city in cities:
        url = f"https://www.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')

        # extract the relevant information
        city_population = city_soup.find(string="Population").find_next("td").get_text()
        city_population_clean = int(city_population.replace(",", ""))
        today = datetime.today().strftime("%d.%m.%Y")

        # keep track of data per city
        population_data.append({"City_name": city,
                        "Population": city_population_clean,
                        "Year_Data_Retrieved": date
                        })

    return pd.DataFrame(population_data)

Does it work?

# population_df

In [102]:
cities = ["Hamburg", "Munich", "San Francisco", "La Paz"]

population_df = populations_dataframe(cities)
population_df

,City_name,Population,Year_Data_Retrieved
0,Hamburg,1964021,2024-12-04
1,Munich,1510378,2024-12-04
2,San Francisco,873965,2024-12-04
3,La Paz,755732,2024-12-04


In [34]:
population_df['Year_Data_Retrieved'][1]

'04.12.2024'

## we have our CITIES DATA FRAME, now we can create sub tables from it to import them to SQL!!

Cities table

*   city_id
*   city_name
*   country_code


---


Population table

*   city_id
*   population
*   timestamp_population


In [37]:
cities_df

,City_name,Country,Latitude,Longitude
0,Hamburg,Germany,53.550000,10.000000
1,Munich,Germany,48.137500,11.575000
2,San Francisco,United States,37.777500,-122.416389
3,La Paz,Bolivia,-16.495833,-68.133333


In [38]:
cities_to_db = cities_df[["City_name", "Country"]]
cities_to_db

,City_name,Country
0,Hamburg,Germany
1,Munich,Germany
2,San Francisco,United States
3,La Paz,Bolivia


In [39]:
population_df

,City_name,Population,Year_Data_Retrieved
0,Hamburg,1964021,04.12.2024
1,Munich,1510378,04.12.2024
2,San Francisco,873965,04.12.2024
3,La Paz,755732,04.12.2024


In [40]:
population_to_db = population_df[["Population", "Year_Data_Retrieved"]]
population_to_db

,Population,Year_Data_Retrieved
0,1964021,04.12.2024
1,1510378,04.12.2024
2,873965,04.12.2024
3,755732,04.12.2024


In [41]:
# install if needed
!pip install sqlalchemy
!pip install pymysql

creating the tables in SQL

USE sql_workshop;

-- Create the 'authors' table

CREATE TABLE cities2_df (
    city_id INT AUTO_INCREMENT, -- Automatically generated ID for each author
    city_name VARCHAR(100) NOT NULL, -- Name of the author
    country_id VARCHAR(100) NOT NULL,
    PRIMARY KEY (city_id) -- Primary key to uniquely identify each author
);



---



-- Create the 'books' table


CREATE TABLE population2_df (
    city_id INT, -- Automatically generated ID for each book
    population_id INT AUTO INCREMENT,
    population INT, -- Title of the book
    timestamp_population INT, -- Year the book was published
    PRIMARY KEY (Population_id),
    FOREIGN KEY (City_id) REFERENCES cities(City_id)
);

Sending our tables to SQL

In [45]:
schema = "gans_local" # The name of your database
host = "127.0.0.1"
user = "root"
password = USE Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [46]:
cities_to_db

,City_name,Country
0,Hamburg,Germany
1,Munich,Germany
2,San Francisco,United States
3,La Paz,Bolivia


Now let's send it

In [48]:
cities_to_db.to_sql("cities",
                  if_exists="append",
                  con=connection_string,
                  index=False)

4

In [49]:
#Read the "cities" table from the database into the notebook
#This step is needed to fetch the "city_id" column and integrate 
#it into the "population_to_db" dataframe. The "city_id" column in "population_to_db" 
#dataframe will serve as a foreign key in order to establish a relation between both
#"cities" & "population" tables

In [50]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,City_id,City_name,Country
0,1,Hamburg,Germany
1,2,Munich,Germany
2,3,San Francisco,United States
3,4,La Paz,Bolivia
4,5,Hamburg,Germany
5,6,Munich,Germany
6,7,San Francisco,United States
7,8,La Paz,Bolivia


In [51]:
population_to_db["City_id"] = cities_from_sql["City_id"]
population_to_db

,Population,Year_Data_Retrieved,City_id
0,1964021,04.12.2024,1
1,1510378,04.12.2024,2
2,873965,04.12.2024,3
3,755732,04.12.2024,4


In [52]:
# Getting the "City_id" to the population df

population_to_db["City_id"] = cities_from_sql["City_id"]
population_to_db

,Population,Year_Data_Retrieved,City_id
0,1964021,04.12.2024,1
1,1510378,04.12.2024,2
2,873965,04.12.2024,3
3,755732,04.12.2024,4


In [53]:
# Push the "population_to_db" to the empty "population" table in the MySQL data base
population_to_db.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

4

In [54]:
population_from_sql = pd.read_sql("population", con=connection_string)
population_from_sql

,Population_id,Population,Year_Data_Retrieved,City_id
0,1,1964021,02.12.2024,1
1,2,1510378,02.12.2024,2
2,3,873965,02.12.2024,3
3,4,755732,02.12.2024,4
4,5,1964021,04.12.2024,1
5,6,1510378,04.12.2024,2
6,7,873965,04.12.2024,3
7,8,755732,04.12.2024,4


Bonus!

LET'S TRY OUR CITY FUNCTION WITH OTHER CITIES!!

In [56]:
# cities_dataframe; is our function for calling the cities, country, lat and lon.

In [57]:
list_of_cities = [ "Hamburg", "Munich", "San Francisco", "La Paz"] #original cities

cities_df = cities_dataframe(list_of_cities)
cities_df

,City_name,Country,Latitude,Longitude
0,Hamburg,Germany,53.550000,10.000000
1,Munich,Germany,48.137500,11.575000
2,San Francisco,United States,37.777500,-122.416389
3,La Paz,Bolivia,-16.495833,-68.133333


In [58]:
list_of_cities = [ "Frankfurt", "Berlin"] #new cities

cities_df = cities_dataframe(list_of_cities)
cities_df

,City_name,Country,Latitude,Longitude
0,Frankfurt,Germany,50.110556,8.682222
1,Berlin,Germany,52.520000,13.405000


Did we check or populations function? let's check it one more time ;) is the function that calls population, date and the city

In [60]:
cities = ["Hamburg", "Munich", "San Francisco", "La Paz"]

population_df = populations_dataframe(cities)
population_df

,City_name,Population,Year_Data_Retrieved
0,Hamburg,1964021,04.12.2024
1,Munich,1510378,04.12.2024
2,San Francisco,873965,04.12.2024
3,La Paz,755732,04.12.2024
